# Import libraries

In [1]:
from dotenv import load_dotenv
import os
from langchain_anthropic import ChatAnthropic
from langchain import hub
from langchain_community.document_loaders import ObsidianLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

# Set variables

In [2]:
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Load, split and store embeddings

In [3]:
loader = ObsidianLoader("/Users/danielmcateer/Library/Mobile Documents/iCloud~md~obsidian/Documents/Ideaverse")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant notes from Obsidian

In [4]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatAnthropic(temperature=0.5, anthropic_api_key=anthropic_api_key, model_name="claude-3-opus-20240229")



/Users/danielmcateer/anaconda3/envs/local-RAG/lib/python3.11/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! api_key is not default parameter.
                api_key was transferred to model_kwargs.
                Please confirm that api_key is what you intended.
  warnings.warn(
/Users/danielmcateer/anaconda3/envs/local-RAG/lib/python3.11/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! retriever is not default parameter.
                retriever was transferred to model_kwargs.
                Please confirm that retriever is what you intended.
  warnings.warn(
/Users/danielmcateer/anaconda3/envs/local-RAG/lib/python3.11/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! prompt is not default parameter.
                prompt was transferred to model_kwargs.
                Please confirm that prompt is what you intended.
  warnings.warn(


ValidationError: 1 validation error for ChatAnthropic
model_name
  field required (type=value_error.missing)

# Run RAG chain using LangChain

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Get a response from the RAG chain

In [17]:
print(rag_chain.invoke("What are the potential future capabilities of AI?"))

The potential future capabilities of AI include advanced audio search functions, utilizing AI as a tool to understand human consciousness better, and the use of code as a source of truth in AI development. Additionally, AI may play a role in addressing the "meaning crisis" by providing accurate information and reducing misinformation.


# Cleanup

In [8]:
vectorstore.delete_collection()